In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import  Dataset
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm
import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import kagglehub
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
import shutil
import glob
import time
import torchvision.utils
from concurrent.futures import ThreadPoolExecutor
import torchvision.models as mdl


import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

writer = SummaryWriter('runs/project')

In [ ]:
# Download the dataset


path_set_1 = kagglehub.dataset_download('vipoooool/new-plant-diseases-dataset')
path_set_2 = kagglehub.dataset_download('amandam1/healthy-vs-diseased-leaf-image-dataset')

path_set_1 = os.path.join(path_set_1, 'New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)')

print (f"Path set 1: '{path_set_1}'")
print (f"Path set 2: {path_set_2}")

## Dataset Hyperparams

In [ ]:
# Trainingsdata params
random_erase=0.1
scale_factor = 28
batch_size = 64
width = 224
height = 224


# Leave this on False ... 70.000 images will crash the RAM
move_to_cuda_preprocess = False


In [ ]:
# Data augmentation and normalization for training
transform = transforms.Compose([
    transforms.Resize((width, height)),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(224, padding=4),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir=".", transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 1]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path)
        label = self.labels_df.iloc[idx, 2] if 'Labels' in self.labels_df.columns else -1
        label = np.float32(label if label == 0 or label == 1 else 0)
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
def generate_csv(root_dir, csv_path):
    data = []
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    index = 0
    num_label_0 = 0
    num_label_1 = 0

    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.JPG', '.JPEG')):
                subfolder_name = os.path.basename(subdir)
                label = 0 if subfolder_name.endswith('healthy') else 1
                if label == 0:
                    num_label_0 += 1
                else:
                    num_label_1 += 1

                # Relative path from root (./train or ./valid)
                rel_path = os.path.relpath(os.path.join(subdir, file), '.')

                data.append([index, rel_path.replace("\\", "/"), label])
                index += 1
    print(f"Number of healthy images: {num_label_0}")
    print(f"Number of diseased images: {num_label_1}")
    df = pd.DataFrame(data, columns=['Index', 'Image', 'Labels'])
    df.to_csv(csv_path, index=False)
    print(f"Saved {len(df)} entries to {csv_path}")

In [ ]:
def split_csv(csv_path, train_csv_path, valid_csv_path, split_ratio=0.8, overwrite=False):
    df = pd.read_csv(csv_path)
    train_df = df.sample(frac=split_ratio, random_state=seed)
    valid_df = df.drop(train_df.index)

    # If train_csv_path exists, append and update index
    if os.path.exists(train_csv_path) and not overwrite:
        existing_train = pd.read_csv(train_csv_path)
        train_df = pd.concat([existing_train, train_df], ignore_index=True)
        train_df['Index'] = range(len(train_df))
    else:
        os.makedirs(os.path.dirname(train_csv_path), exist_ok=True)
        train_df['Index'] = range(len(train_df))

    # If valid_csv_path exists, append and update index
    if os.path.exists(valid_csv_path) and not overwrite:
        existing_valid = pd.read_csv(valid_csv_path)
        valid_df = pd.concat([existing_valid, valid_df], ignore_index=True)
        valid_df['Index'] = range(len(valid_df))
    else:
        valid_df['Index'] = range(len(valid_df))
        os.makedirs(os.path.dirname(valid_csv_path), exist_ok=True)

    train_df.to_csv(train_csv_path, index=False)
    valid_df.to_csv(valid_csv_path, index=False)

    print(f"Saved {len(train_df)} training entries to {train_csv_path}")
    print(f"Saved {len(valid_df)} validation entries to {valid_csv_path}")


In [ ]:
def evaluate_dataset(csv_path_1, csv_path_2, model):
    
    model.eval()
    dataset1 = CustomDataset(csv_file=csv_path_1, transform=transform)
    dataset2 = CustomDataset(csv_file=csv_path_2, transform=transform)

    loader1 = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=False)
    loader2 = torch.utils.data.DataLoader(dataset2, batch_size=64, shuffle=False)

    def extract_embeddings(loader):
        embeddings = []
        with torch.no_grad():
            for images, _ in tqdm(loader, desc="Extracting embeddings"):
                images = images.to(device)
                feats = model(images)
                embeddings.append(feats.cpu())
        return torch.cat(embeddings, dim=0)

    embeddings1 = extract_embeddings(loader1)
    embeddings2 = extract_embeddings(loader2)
    min_len = min(len(embeddings1), len(embeddings2))
    embeddings1 = embeddings1[:min_len]
    embeddings2 = embeddings2[:min_len]

    def calculate_fid(mu1, sigma1, mu2, sigma2, eps=1e-6):
        # Calculate the mean and covariance difference
        diff = mu1 - mu2
        covmean, _ = sqrtm(sigma1 @ sigma2, disp=False)

        # Numerical stability
        if np.iscomplexobj(covmean):
            covmean = covmean.real

        fid = diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)
        return fid

    # Convert to numpy
    embeddings1_np = embeddings1.numpy()
    embeddings2_np = embeddings2.numpy()

    # Compute mean and covariance
    mu1 = np.mean(embeddings1_np, axis=0)
    mu2 = np.mean(embeddings2_np, axis=0)
    sigma1 = np.cov(embeddings1_np, rowvar=False)
    sigma2 = np.cov(embeddings2_np, rowvar=False)

    # Calculate FID
    fid_score = calculate_fid(mu1, sigma1, mu2, sigma2)
    print("FID Score:", fid_score)

    all_embeddings = torch.cat([embeddings1, embeddings2], dim=0).numpy()
    labels = np.array([0]*len(embeddings1) + [1]*len(embeddings2))

    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(all_embeddings)

    plt.figure(figsize=(8,6))
    plt.scatter(embeddings_2d[labels==0,0], embeddings_2d[labels==0,1], alpha=0.5, label='Dataset 1')
    plt.scatter(embeddings_2d[labels==1,0], embeddings_2d[labels==1,1], alpha=0.5, label='Dataset 2')
    plt.legend()
    plt.title("t-SNE of Dataset Embeddings")
    plt.show()

In [ ]:


# Run for both train and valid
generate_csv(os.path.join(path_set_1, './train'), 'data/train.csv')
generate_csv(os.path.join(path_set_1, './valid'), 'data/valid.csv')
generate_csv(path_set_2, 'data/dataset_2.csv')
os.makedirs('./data/evaluate', exist_ok=True)
shutil.copy('data/valid.csv', 'data/evaluate/valid.csv')
split_csv('data/dataset_2.csv', 'data/train_2.csv', 'data/test.csv')
split_csv('data/train_2.csv', 'data/train.csv', 'data/valid.csv')

df = pd.read_csv('data/train.csv')
min_count = df['Labels'].value_counts().min()

# Sample min_count from each class
df_minority = df[df['Labels'] == df['Labels'].value_counts().idxmin()]
df_majority = df[df['Labels'] == df['Labels'].value_counts().idxmax()].sample(min_count, random_state=42)

# Combine and shuffle
df_balanced = pd.concat([df_minority, df_majority]).sample(frac=1, random_state=42).reset_index(drop=True)
df_balanced.to_csv('data/train_undersampled.csv', index=False)


In [ ]:


def process_and_save_image(idx, row, transform, output_dir):
    img_path = os.path.join('.', row['Image'])
    try:
        image = Image.open(img_path).convert('RGB')
        processed_image = transform(image)  # Tensor C,H,W
        label = row['Labels'] if not pd.isna(row['Labels']) else -1
        label = np.float32(label if label == 0 or label == 1 else 0)

        # Convert tensor back to PIL Image for saving
        to_pil = transforms.ToPILImage()
        pil_img = to_pil(processed_image)

        # Save processed image with zero-padded index filename
        out_name = f"{idx:06d}.png"
        out_path = os.path.join(output_dir, out_name)
        pil_img.save(out_path)

        return pil_img, label
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None, None
    
def preprocess_and_save_images(csv_file, output_dir, output_csv, transform, num_workers=8):
    if os.path.exists(output_csv):
        print(f"Output CSV '{output_csv}' already exists. Skipping preprocessing.")
        return

    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_file)
    processed_entries = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [
            executor.submit(process_and_save_image, idx, row, transform, output_dir)
            for idx, row in df.iterrows()
        ]

        # Wrap the futures list with tqdm, then call result() inside the loop
        for f in tqdm(futures, desc="Preprocessing & saving images"):
            out_name, label = f.result()
            if out_name is not None:
                processed_entries.append([out_name, label])

    processed_df = pd.DataFrame(processed_entries, columns=['Image', 'Labels'])
    processed_df.to_csv(output_csv, index=False)
    print(f"Saved processed images to {output_dir} and CSV to {output_csv}")


In [ ]:
class CustomProcessedDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file):
        self.labels_df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_path = self.labels_df.iloc[idx]['Image']
        # If saved as PNG, just load and convert to tensor
        image = Image.open(img_path).convert('RGB')
        image = transforms.ToTensor()(image)  # Only if needed, otherwise remove
        label = np.float32(self.labels_df.iloc[idx]['Labels'])
        return image, label

In [ ]:
# model = models.resnet50(pretrained=True)
# model.fc = torch.nn.Identity()
# model = model.to(device)
# evaluate_dataset('data/evaluate/valid.csv', 'data/dataset_2.csv', model)


In [ ]:
# Define the dataset and data loaders
preprocess_and_save_images('data/train_undersampled.csv', 'data/processed_images/train', 'data/processed_train.csv', transform, num_workers=16)
preprocess_and_save_images('data/valid.csv', 'data/processed_images/valid', 'data/processed_valid.csv', transform, num_workers=16)

train_dataset = CustomProcessedDataset(csv_file='data/processed_train.csv')
val_dataset = CustomProcessedDataset(csv_file='data/processed_valid.csv')
test_dataset = CustomDataset(csv_file='data/test.csv', transform=transform)

In [ ]:
def get_gpu_dataloader(sampler, dataset,batch_size=1, shuffle=False):
    data = []
    labels = []
    for img, label in tqdm(dataset):
        data.append(img)
        labels.append(label)

    data = torch.stack(data)
    labels = torch.tensor(labels)

    data = data.to(device)
    labels = labels.to(device)

    loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(data, labels),
    batch_size=batch_size, shuffle=shuffle, sampler=sampler)
    return loader

In [ ]:
def get_sampler(dataset, weighted=False):
    if weighted:
        # WeightedRandomSampler against class imbalance
        labels = dataset.labels_df['Labels'].values
        class_sample_count = np.array([len(np.where(labels == t)[0]) for t in np.unique(labels)])
        weights = 1. / class_sample_count
        samples_weights = np.array([weights[int(t)] for t in labels])
        samples_weights = torch.from_numpy(samples_weights).float()

        return torch.utils.data.WeightedRandomSampler(samples_weights, len(samples_weights), replacement=True)
    else:
        return torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=len(dataset))
 

# Data loaders
train_sampler = get_sampler(train_dataset)
validation_sampler = get_sampler(val_dataset)

training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=16)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, sampler=validation_sampler, num_workers=16)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Data exploration

In [ ]:

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_dataset)
next(dataiter)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(labels)

print(f"Number of samples in train_dataset: {len(train_dataset)}")
print(f"Number of samples in val_dataset_plant: {len(val_dataset)}")
print(f"Batch size: {batch_size}")
print(f"Image shape (C, H, W): {images.shape}")
print(f"Label example: {labels}")

In [ ]:
# Datenexploration: Qualität, Bitrate, Verteilung der Labels, Dimension, etc.

# Verteilung der Labels im Trainings- und Validierungsdatensatz
train_labels = train_dataset.labels_df['Labels']
val_labels = val_dataset.labels_df['Labels']

print("Train label distribution:")
print(train_labels.value_counts())
print("\nValidation label distribution:")
print(val_labels.value_counts())

# Plot label distribution
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
train_labels.value_counts().plot(kind='bar', title='Train Label Distribution')
plt.subplot(1, 2, 2)
val_labels.value_counts().plot(kind='bar', title='Validation Label Distribution')
plt.tight_layout()
plt.show()

# Show label distribution in a batch from the weighted sampler
def plot_batch_label_distribution(loader, name):
    batch_imgs, batch_labels = next(iter(loader))
    unique, counts = np.unique(batch_labels.cpu().numpy(), return_counts=True)
    plt.bar(unique, counts, tick_label=[str(int(u)) for u in unique])
    plt.title(f"{name} batch label distribution")
    plt.xlabel("Label")
    plt.ylabel("Count")
    plt.show()

print("Weighted sampler batch label distribution (train):")
plot_batch_label_distribution(training_loader, "Train")
print("Weighted sampler batch label distribution (validation):")
plot_batch_label_distribution(validation_loader, "Validation")

# Beispielbild: Qualität, Dimension, Bitrate
sample_img_path = train_dataset.labels_df.iloc[0]['Image']
sample_img = Image.open(sample_img_path)
print(f"Sample image path: {sample_img_path}")
print(f"Image mode: {sample_img.mode}")
print(f"Image size (W x H): {sample_img.size}")
print(f"Image format: {sample_img.format}")

# Bitrate (Dateigröße in KB)
img_size_kb = os.path.getsize(sample_img_path) / 1024
print(f"Image file size: {img_size_kb:.2f} KB")

# Check for class imbalance
imbalance_ratio = train_labels.value_counts().min() / train_labels.value_counts().max()
print(f"Imbalance ratio (min/max): {imbalance_ratio:.3f}")
if imbalance_ratio < 0.5:
    print("Warning: The dataset is imbalanced!")

# Data augmentation check
print("\nData augmentation in use:")
for t in transform.transforms:
    print("-", t)

## Models

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4, 4),

            nn.Conv2d(16, 8, kernel_size=15, padding=7),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4, 4),

        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(8 * int(width/(2**7)) * int(height/(2**7)), 4 * int(width/(2**7)) * int(height/(2**7))),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4 * int(width/(2**7)) * int(height/(2**7)), num_classes),
            #nn.Sigmoid(),
            nn.Softmax(),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x #x.reshape(-1)

## Training

In [ ]:
def log_metrics_to_tensorboard(model_name, epoch, test_loss):
    writer.add_scalar(f'{model_name}/test_loss', test_loss, epoch)

def log_accuracy_to_tensorboard(model_name, epoch, accuracy):
    writer.add_scalar(f'{model_name}/accuracy', accuracy, epoch)

In [ ]:
def build_config(model, criterion=nn.BCELoss(), learn_rate=0.0001, optim_base=optim.AdamW, epochs=150, patience=10, early_stopping=True,choose_best_loss_model=True):
    dic = {}
    dic["model"] = model#copy.deepcopy(model)
    if criterion: dic["criterion"] = criterion
    if learn_rate: dic["learn_rate"] = learn_rate
    if optim_base: dic["optim_base"] = optim_base
    if epochs: dic["epochs"] = epochs
    if early_stopping: dic["early_stopping"] = early_stopping
    if choose_best_loss_model: dic["choose_best_loss_model"] = choose_best_loss_model
    return dic 

def simple_train(model, train_loader, val_loader, criterion=nn.BCELoss(), learn_rate=0.0001, optim_base=optim.AdamW, epochs=150, patience=10, early_stopping=True,choose_best_loss_model=True):
    # Calculate the total number of parameters
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(f'Total number of parameters: {total_params}')
    intro_text = f"""Training {total_params} Parameters {model._get_name()} with 
          criterion: {criterion},
          optim_base: {optim_base},
          learn_rate: {learn_rate}, 
          epochs: {epochs}, 
          patience: {patience}, 
          early_stopping: {early_stopping}, 
          choose_best_loss_model: {choose_best_loss_model}"""
    print(intro_text)
    
    print("--------")

    hash_int = hash((model._get_name(), criterion, learn_rate, optim_base, epochs, patience, early_stopping, choose_best_loss_model))
    writer.add_text(f'{model._get_name()}_{hash_int}/train_loss', intro_text)

    optimizer = optim_base(model.parameters(), lr=learn_rate)
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.75)

    # Move the model to GPU if available
    model.to(device)

    # Training loop with early stopping
    best_val_loss = float('inf')
    best_val_acc = 0
    patience_counter = 0
    outer_loop = tqdm(range(epochs), desc="Epochs")
    
    for epoch in outer_loop:
        #print(f'Epoch {epoch + 1}/{epochs}')
        model.train()
        running_loss = 0.0
        for idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            
            if (outputs.shape[1] == 2):
                labels = labels.long()
            

            outputs = outputs.squeeze(-1)

            loss = criterion(outputs, labels)

            writer.add_scalar(f'{model._get_name()}_{hash_int}/train_loss', loss.detach().item(), epoch*len(train_loader)+idx)
            running_loss += loss.detach().item()

            # Backward pass
            loss.backward()
            optimizer.step()

        #scheduler.step()
        #print(f'Training loss: {running_loss / len(train_loader)}')
        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                #print(outputs.shape)
                if (outputs.shape[1] == 2):
                    labels = labels.long()

                outputs = outputs.squeeze(-1)


                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Convert the outputs to predicted class labels
                if (outputs.shape[1] == 2):
                    predicted = outputs.argmax(dim=1).float() #torch.max(outputs, 1)
                else:
                    predicted = (outputs>0.5).float() #torch.max(outputs, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total

        writer.add_scalar(f'{model._get_name()}_{hash_int}/running_loss', running_loss / len(train_loader), epoch)
        writer.add_scalar(f'{model._get_name()}_{hash_int}/val_loss', val_loss, epoch)
        writer.add_scalar(f'{model._get_name()}_{hash_int}/val_acc', val_accuracy, epoch)

        tqdm.set_description_str(outer_loop,f"Epochs: [acc: ({val_accuracy:.4f}/{best_val_acc:.4f}), loss: ({val_loss:.4f}/{best_val_loss:.4f})]")
        #print(f'Validation loss: {val_loss / len(val_loader)}')
        #print(f'Validation accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = patience_counter - 1 if patience_counter > 0 else 0
            torch.save(model.state_dict(), 'best_acc_model.pth')
            #print(f'Best model saved with valid acc of {val_accuracy:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_val_model.pth')
            #print(f'Best model saved with valid loss of {val_loss / len(val_loader)}')
        else:
            patience_counter += 1
            if patience_counter >= patience and early_stopping:
                print("Early stopping triggered")
                break
    if (choose_best_loss_model):
        model.load_state_dict(torch.load('best_val_model.pth'))
        return model
    return model

In [ ]:
def eval(model, test_loader):
    model.to(device)

    # Make predictions on the test dataset
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for images, label in test_loader:
            images = images.to(device)
            outputs = model(images)
            if (outputs.shape[1] == 2):
                predicted = outputs.argmax(dim=1).float()
            else:
                predicted = (outputs>0.5).float()
            labels.extend(label.cpu().numpy())
            predictions.extend(predicted.cpu().numpy())

    # Create a DataFrame with IDs and predictions
    submission_df = pd.DataFrame({
        #'images': test_dataset.labels_df['Images'],
        'labels': predictions,
        'groundtruth': labels
    })

    submission_df_ones = submission_df[submission_df.groundtruth == 1]
    submission_df_zeros = submission_df[submission_df.groundtruth == 0]
    tp = (submission_df_ones.labels == submission_df_ones.groundtruth).sum()
    tn = (submission_df_zeros.labels == submission_df_zeros.groundtruth).sum()
    fn = (submission_df_ones.labels != submission_df_ones.groundtruth).sum()
    fp = (submission_df_zeros.labels != submission_df_zeros.groundtruth).sum()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    f1 = (2*precision*recall)/(precision+recall)
    return accuracy, precision, recall, f1, (tp, tn, fn, fp)

In [ ]:
def simple_hyper_train(configurations):
    model_list = []
    for config in configurations: 
        config["train_loader"] = training_loader
        config["val_loader"] = validation_loader
        model = simple_train(**config)
        model_list.append(model)
    results = []
    for model in model_list:
        accuracy, precision, recall, f1, (tp, tn, fn, fp) = eval(model, test_loader)
        result = {
            "model": model,
            "accuracy": accuracy, 
            "precision": precision,
            "recall": recall, 
            "f1": f1, 
            "hidden": (tp, tn, fn, fp),
        }
        results.append(result)
    return results


In [ ]:
resnet = mdl.resnet18(weights="IMAGENET1K_V1")
resnet.fc = nn.Linear(resnet.fc.in_features, 2)

alexnet = mdl.alexnet(weights="IMAGENET1K_V1")
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 2)




In [ ]:
def unreg_params(model):
    for param in model.parameters():
        param.requires_grad = False



resnet_last = mdl.resnet18(weights="IMAGENET1K_V1")
unreg_params(resnet_last)
resnet_last.fc = nn.Linear(resnet_last.fc.in_features, 2)

alexnet_last = mdl.alexnet(weights="IMAGENET1K_V1")
unreg_params(alexnet_last)
alexnet_last.classifier[6] = nn.Linear(alexnet_last.classifier[6].in_features, 2)


In [ ]:
configs = [
    build_config(resnet,criterion=nn.CrossEntropyLoss(), learn_rate=0.01,epochs=75),
    build_config(alexnet,criterion=nn.CrossEntropyLoss(), learn_rate=0.01,epochs=75),

    build_config(CNN(),criterion=nn.CrossEntropyLoss(), learn_rate=0.01,epochs=75),

    build_config(resnet_last,criterion=nn.CrossEntropyLoss(), learn_rate=0.01,epochs=75),
    build_config(alexnet_last,criterion=nn.CrossEntropyLoss(), learn_rate=0.01,epochs=75),
    
]

In [ ]:
def shortend(configs):
    conf_out = []
    for config in configs:
        d = {
            "name": config["model"]._get_name(),
            "accuracy": float(config["accuracy"]),
            "f1": float(config["f1"])
        }
        conf_out.append(d)
    return conf_out

In [ ]:
results = simple_hyper_train(configs)
shortend(results)

In [ ]:

def inference_time(model):
    image, label = val_dataset[0]
    image = image.unsqueeze(0)
    model.cpu()
    since = time.time()
    model(image)
    now = time.time()
    now - since

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs